<a href="https://colab.research.google.com/github/minsangggg/Machine_Running-Ai-/blob/main/Study/19.%EC%8B%A0%EA%B2%BD%EB%A7%9D%20%EA%B5%AC%EC%A1%B0(%EB%94%A5%EB%9F%AC%EB%8B%9D).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
tf.__version__

2025-09-02 15:30:30.912308: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


'2.20.0'

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
import sklearn
from sklearn import datasets
iris=datasets.load_iris()
iris

ModuleNotFoundError: No module named 'sklearn'

In [6]:
X_iris=iris.data
y_iris=iris.target

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X_iris, y_iris, test_size=0.3,random_state=1)

NameError: name 'iris' is not defined

In [ ]:
from sklearn.neural_network import MLPClassifier
mlp=MLPClassifier(hidden_layer_sizes=(50,30), activation='logistic',
                  solver='adam',max_iter=1000)

In [ ]:
mlp.fit(X_train, y_train)

MLPClassifier(activation='logistic', hidden_layer_sizes=(50, 30), max_iter=1000)

In [ ]:
mlp.score(X_test, y_test)

1.0

In [ ]:
import sklearn
sklearn.neural_network.MLPClassifier(

hidden_layer_sizes=(100,), activation='relu', solver='adam', alpha=0.0001,

batch_size='auto', learning_rate='constant', learning_rate_init=0.001,
power_t=0.5, max_iter=200, shuffle=True, random_state=None, tol=0.0001,
verbose=False, momentum=0.9, nesterovs_momentum=True,
early_stopping=False, validation_fraction=0.1, beta_1=0.9, beta_2=0.999,
epsilon=1e-08, n_iter_no_change=10
)

MLPClassifier()

In [ ]:
import tensorflow.compat.v1 as tf  # TF 1.x API를 그대로 쓸 수 있도록 제공되는 호환 모듈.
tf.disable_v2_behavior()  #TensorFlow 2.x의 기능을 꺼버리고, 1.x 방식으로 동작하기.

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
X=tf.placeholder(tf.float32,[None,4])  #4는 특징개수
y=tf.placeholder(tf.float32,[None,3])

## W가중치, b(편향) - 1 layer


In [ ]:
W1= tf.Variable(tf.random_normal([4,50],dtype=tf.float32))#가중치 행렬= 평균 0, 분산 1의 정규분포로 초기화([입력차원, 출력 뉴런 개수])
b1=tf.Variable(tf.zeros([50],dtype=tf.float32))#bias벡터= 0으로 초기화([크기]) ->은닉층 뉴런마다 하나씩 bias 추가
h1=tf.nn.sigmoid(tf.matmul(X,W1)+b1)# 은닉층=활성화함수sigmoid(입력X과 가중치W1의 행렬곱+각 뉴런에 bias 더함) -> [None,50]

## 2layer - 30노드로 구성

In [ ]:
W2= tf.Variable(tf.random_normal([50,30])) # 앞 층(뉴런 50개)-> 다음 층(뉴런 30개) 연결
b2= tf.Variable(tf.zeros([30])) # 0을 30개 만든다
h2= tf.nn.sigmoid(tf.matmul(h1,W2)+b2) #두번째 은닉층=시그모이드 함수(첫 번째 은닉층 [None,50],가중치크기[50,30]+bias벡터)

## 3layer

In [ ]:
W3= tf.Variable(tf.random_normal([30,3])) #가중치 행렬= 앞 층 뉴런30, 출력뉴런3
b3= tf.Variable(tf.zeros([3]))            #bias 벡터
h3= tf.nn.softmax(tf.matmul(h2,W3)+b3)    #출력층; 최종 출력[None,3] 세 값의 합은 항상 1

In [ ]:
cross_entropy = -tf.reduce_sum(y*tf.log(h3), reduction_indices=[1]) #손실함수= 샘플별 손실계산(정답인 위치의 확률만, axis=1)
loss = tf.reduce_mean(cross_entropy) #전체 batch에 대해 평균을 낸 손실값
train = tf.train.AdamOptimizer().minimize(loss) #Adam Optimizer: 경사하강법(Gradient Descent) 알고리즘의 개선된 버전

In [ ]:
import pandas as pd

feed_train = {X: X_train, y: pd.get_dummies(y_train).values} #X:입력 데이터 , y:원-핫 인코딩된 정답 라벨

init = tf.global_variables_initializer() #모든 변수 초기화
sess = tf.Session() #W1, b1, W2, b2, W3, b3 같은 가중치와 bias들이 처음 값(랜덤, 0 등)으로 세팅됨
sess.run(init)

for i in range(1000):
    sess.run(train, feed_dict=feed_train)  #데이터를 넣고 학습(가중치 업데이트) 실행

    if i%100 == 0:     #100step마다 현재 손실(loss) 확인
        tr_loss = sess.run(loss, feed_dict=feed_train)
        print("Step %d, %.5f" % (i, tr_loss)) #step이 늘수록 손실이 줄어듦
        

Step 0, 2.14222
Step 100, 0.47408
Step 200, 0.24292
Step 300, 0.16031
Step 400, 0.11676
Step 500, 0.09678
Step 600, 0.08406
Step 700, 0.07575
Step 800, 0.06982
Step 900, 0.06547


In [ ]:
correct_prediction = tf.equal(tf.argmax(h3, 1), tf.argmax(y, 1)) #(예측 클래스; 가장 확률 높은 index),실제 정답 클래스(원핫인코딩->index 변환)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)) #맞춘 비율
feed_test = {X: X_test, y: pd.get_dummies(y_test).values} #테스트 데이터셋 준비
print(sess.run(accuracy, feed_dict=feed_test))

1.0


In [ ]:
sess.close()

# Keras로 신경망 구성

In [ ]:
#iris dataset load
from sklearn import datasets
iris = datasets.load_iris()
X = iris.data
y = iris.target
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [ ]:
from tensorflow.keras.models import Sequential
model=Sequential()  #Sequential이란 모델 구조 생성
model

<Sequential name=sequential_1, built=False>

In [ ]:
from tensorflow.keras.layers import Input, Dense

model.add(Input(shape=(4,))) #입력 특성이 4개
model.add(Dense(50, activation='sigmoid'))  #첫번째 은닉층, 뉴런 50개
model.add(Dense(50, activation='sigmoid'))  #두번째 은닉층
model.add(Dense(50, activation='softmax'))  #출력층
#무조건 마지막 층이 출력층

In [ ]:
model

<Sequential name=sequential_1, built=True>

In [ ]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:

model.fit(X_train, y_train, epochs=300, verbose=1)

Epoch 1/300


RuntimeError: `tf.data.Dataset` only supports Python-style iteration in eager mode or within tf.function.

In [ ]:
model.evaluate(X_test, y_test)

RuntimeError: `tf.data.Dataset` only supports Python-style iteration in eager mode or within tf.function.

In [ ]:
sess.close()

Pytorch로 구성

In [ ]:
import torch
torch.__version__

'2.8.0+cu126'

In [ ]:
if torch.cuda.is_available():   #Pytorch가 GPU를 사용할 수 있는지 확인
  torch.cuda.get_device_name(0) # 첫 번째 GPU(0번째)의 이름 출력

In [ ]:
from sklearn import datasets
iris=datasets.load_iris()
X_iris=iris.data
y_iris=iris.target

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X_iris, y_iris, test_size=0.3,random_state=1)

In [ ]:
import torch

X_train=torch.tensor(X_train, dtype=torch.float32) #torch.tensor: PyTorch의 Tensor객체로 변환
y_train=torch.tensor(y_train, dtype=torch.float32)
X_test=torch.tensor(X_test, dtype=torch.float32)
y_test=torch.tensor(y_test, dtype=torch.float32)

In [ ]:
import torch.nn as nn
model = nn.Sequential(
    nn.Linear(4,50), nn.Sigmoid(),   #입력4차원 ->출력 50차원
    nn.Linear(50,30), nn.Sigmoid(),
    nn.Linear(30,3), nn.Softmax(dim=1), #출력값을 확률처럼 변환, 각 샘플별 클래스 확률 합이 1이 되도록 만듦
)

In [ ]:
import torch.optim as optim

#w의 초기값 설정
for layer in model:
  if isinstance(layer, nn.Linear):
    nn.init.xavier_uniform_(layer.weight) ##nn.Linear층이면 xavier 초기화

#손실함수
crit = nn.CrossEntropyLoss() #다중 클래스 분류에 사용하는 손실 함수
#옵티마이저
opt = optim.Adam(model.parameters()) #모델의 학습 가능한 모든 파라미터(가중치, bias)를 옵티마이저에 전달

In [ ]:
y_train = y_train.long()

In [ ]:
for epoch in range(300):
  outputs=model(X_train)  #학습 데이터 모델에 넣음
  loss=crit(outputs, y_train) #손실계산= 모델 예측 vs정답(y_train) 비교 
  opt.zero_grad() #PyTorch는 기울기(gradient)를 계속 누적하므로, 매 학습 step마다 초기화 필요
  loss.backward() #역전파 수행, 가중치마다 손실에 대한 기울기를 계산
  opt.step() #옵티마이저 가중치 업데이트


In [ ]:
with torch.no_grad(): #평가 시에는 기울기(gradient) 계산 불필요
  test_outputs= model(X_test) #1. 테스트 데이터 예측
  _, predicted=torch.max(test_outputs,1) #2. 가장 큰 값과 인덱스를 반환, 가장 높은 확률 높은 클래스 선택
  accuracy=(predicted==y_test).sum().item()/y_test.size(0) #정확도= 맞춘것.맞춘개수/전체 샘플 수
  print(accuracy)

0.9777777777777777


In [ ]:
sess.close()

# 함수형 API 이용

In [ ]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.layers import concatenate, Activation

In [ ]:
from sklearn import datasets
iris=datasets.load_iris()
iris

X_iris=iris.data
y_iris=iris.target

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X_iris, y_iris, test_size=0.3,random_state=1)

In [ ]:
from typing_extensions import Concatenate

input = Input(shape=(4,))
dense1 = Dense(50, activation='relu')(input)
dense2 = Dense(80, activation='relu')(input)
dense3 = Dense(30, activation='relu')(input)    #입력을 받아서 세개의 다른 Dense층으로 각각 전달
x = concatenate([dense1, dense2, dense3]) #세 갈래의 출력을 하나로 이어 붙임, 최종shape160
output = Dense(3, activation='softmax')(x) #합쳐진 160차원 출력을 최종 출력층(3개 클래스)으로 매핑
                                           #softmax는 다중 클래스 분류
model = Model(inputs=input, outputs=output) #Model()로 입력과 출력을 지정해서 전체 모델 정의
model.summary()  #층별 파라미터 수와 구조 확인

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 4)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 50)        │        250 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 80)        │        400 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 30)        │        150 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 160)       │          0 │ dense_4[0][0],    │
│ (Concatenate)       │                   │            │ dense_5[0][0],    │
│                     │                   │            │ dense_6[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 3)         │        483 │ concatenate_1[0]… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,283 (5.01 KB)

 Trainable params: 1,283 (5.01 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(
    optimizer='adam',  #가중치를 업데이트하는 방법(최적화 알고리즘)
    loss='sparse_categorical_crossentropy', #모델이 얼마나 틀렸는지 계산, 라벨이 이미 원핫 벡터일때 categorical_crossentropy 씀
    metrics=['accuracy']
)#모델이 어떻게 학습할지 정의


In [ ]:
history = model.fit(X_train, y_train, epochs=300, verbose=1)
history

Epoch 1/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.3496 - loss: 2.1594
Epoch 2/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.3705 - loss: 1.3698
Epoch 3/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3350 - loss: 1.0412
Epoch 4/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8054 - loss: 0.8587
Epoch 5/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6761 - loss: 0.8387
Epoch 6/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4447 - loss: 0.8332
Epoch 7/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5609 - loss: 0.8076
Epoch 8/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6854 - loss: 0.7683
Epoch 9/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.6875 - loss: 0.7261 
Epoch 10/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.7073 - loss: 0.6873
Epoch 11/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7448 - loss: 0.6519
Epoch 12/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7187 - l

In [ ]:
loss, acc = model.evaluate(X_test, y_test)
loss,acc   #학습하지 않은 데이터에 대해 손실값과 정확도 계산


#테스트 데이터셋이 2개의 batch로 나누어 평가되었고
# 배치 하나를 처리하는데 걸린시간 35ms
#손실값=0.0844(작을수록 좋음)
#손실 0.0793

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 1.0000 - loss: 0.0844 


(0.07931581139564514, 1.0)

## 예측

In [ ]:
import numpy as np
pred_proba=model.predict(X_test[:5]) #앞 5개 데이터 샘플 예측 수행
pred_class=np.argmax(pred_proba, axis=1) #확률 중 가장 큰 값의 위치(클래스) 선택
pred_class

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step


array([0, 1, 1, 0, 2])

## 평가결과

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
y_pred=model.predict(X_test).argmax(axis=1)
print(classification_report(y_test,y_pred))

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        14
           1       1.00      1.00      1.00        18
           2       1.00      1.00      1.00        13

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



## 모델의 저장

In [ ]:
model.save('iris_model.keras')

# 모델의 로드

In [ ]:
from tensorflow.keras.models import load_model

model = load_model('/content/iris_model.keras')
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 4)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 50)        │        250 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 80)        │        400 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 30)        │        150 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 160)       │          0 │ dense_4[0][0],    │
│ (Concatenate)       │                   │            │ dense_5[0][0],    │
│                     │                   │            │ dense_6[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 3)         │        483 │ concatenate_1[0]… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 3,851 (15.05 KB)

 Trainable params: 1,283 (5.01 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,568 (10.04 KB)

In [ ]:


json_str = model.to_json()
json_str

'{"module": "keras.src.models.functional", "class_name": "Functional", "config": {"name": "functional_1", "trainable": true, "layers": [{"module": "keras.layers", "class_name": "InputLayer", "config": {"batch_shape": [null, 4], "dtype": "float32", "sparse": false, "ragged": false, "name": "input_layer_1"}, "registered_name": null, "name": "input_layer_1", "inbound_nodes": []}, {"module": "keras.layers", "class_name": "Dense", "config": {"name": "dense_4", "trainable": true, "dtype": {"module": "keras", "class_name": "DTypePolicy", "config": {"name": "float32"}, "registered_name": null}, "units": 50, "activation": "relu", "use_bias": true, "kernel_initializer": {"module": "keras.initializers", "class_name": "GlorotUniform", "config": {"seed": null}, "registered_name": null}, "bias_initializer": {"module": "keras.initializers", "class_name": "Zeros", "config": {}, "registered_name": null}, "kernel_regularizer": null, "bias_regularizer": null, "kernel_constraint": null, "bias_constraint":

In [ ]:

import json

json_dic = json.loads(json_str)
with open('iris_model.json', 'w') as f:
  json.dump(json_dic, f, indent=4)